**Installing Dependencies**

In [1]:
!pip install llama-parse
!pip install llama-index-vector-stores-pinecone
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install pinecone
!pip install torchvision
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 50.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cach

#### Import required libraries 

In [2]:
import os
import time
import torch
import pinecone
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.huggingface import HuggingFaceLLM
from sentence_transformers import SentenceTransformer
from llama_index.core import (
    PromptTemplate,
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)

from transformers import pipeline
from llama_index.core.schema import Document
from llama_parse import LlamaParse
import pinecone
from llama_index.vector_stores.pinecone import PineconeVectorStore

from pinecone import Pinecone, ServerlessSpec


from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.service_context import ServiceContext, set_global_service_context




/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


**Defining LLAMA API Key and PINECONE VECTOR Database**

In [3]:
os.environ["PINECONE_API_KEY"] = "d5f546cf-87c9-4a93-8ca1-af1030d5c37c"
PINECONE_API_KEY = "d5f546cf-87c9-4a93-8ca1-af1030d5c37c"
LLAMA_CLOUD_API_KEY = "llx-Jce36FVDag4H86sqG3OfuaII5O0mB3OwMo4HcrTGtLgdbYyY"
os.environ["LLAMA_CLOUD_API_KEY"]="llx-Jce36FVDag4H86sqG3OfuaII5O0mB3OwMo4HcrTGtLgdbYyY"



pc = Pinecone(
        api_key=PINECONE_API_KEY,
        environment = "us-east-1"
)


**Defining LLMs and Embeeding Models**

In [4]:
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

Settings.embed_model =embed_model
context_str ="You are Q & A Bot"
#query_str="Who is Anna Data"
from llama_index.core.prompts.prompts import SimpleInputPrompt

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."

llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    #query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",

    model_kwargs={"torch_dtype": torch.bfloat16}
)

Settings.llm=llm

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [5]:
import nest_asyncio; nest_asyncio.apply()

#### Loading PDF Document using Llama Parser 

In [6]:

parser2 = LlamaParse(
    api_key=LLAMA_CLOUD_API_KEY,
    result_type="text",
    verbose=False,
    language='en'
)
documents = parser2.load_data("Assignment Support Document.pdf")

print("PDF Content Length ",{len(documents)})



PDF Content Length  {32}


**Splitting Document into Chunks/Nodes using Sentence Splitter Embedding**

In [7]:
#Creating Nodes
# Initialize the parser
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
model.save('.\\sentence-transformers',)

#Load the embedding model
local_model_path = ".\\sentence-transformers"
Settings.embed_model = HuggingFaceEmbedding(model_name=local_model_path)

embed_model = HuggingFaceEmbedding(
    model_name=local_model_path
)
parser = SentenceSplitter.from_defaults(chunk_size=512, chunk_overlap=20)

# Parse documents into nodes or chunks
nodes = parser.get_nodes_from_documents(documents)
print(f"Number of nodes created: {len(nodes)}")

# Filter out empty nodes
filtered_nodes = [node for node in nodes if node.get_content() != ""]

print(f"No. of Non Empty Nodes: {len(filtered_nodes)}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of nodes created: 32
No. of Non Empty Nodes: 30


**Creating INDEX using Nodes parsed by Sentence Splitter**

In [8]:
!pip install pinecone-client
from llama_index.vector_stores.pinecone import PineconeVectorStore

from pinecone import Pinecone, ServerlessSpec

os.environ["PINECONE_API_KEY"] = "d5f546cf-87c9-4a93-8ca1-af1030d5c37c"
PINECONE_API_KEY = "d5f546cf-87c9-4a93-8ca1-af1030d5c37c"

pc = Pinecone(
        api_key="d5f546cf-87c9-4a93-8ca1-af1030d5c37c",
        environment = "us-east-1"
)
from pinecone import ServerlessSpec

cloud = 'aws'
region = 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)

index_name = "nagp"
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

dimensions = 768
pc.create_index(name=index_name, dimension=dimensions, metric="cosine", spec=spec)

# Wait for the index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)
print("INdex created", index)





INdex created <pinecone.data.index.Index object at 0x7b6270d7b850>


**Upserting Nodes Created into Pinecone Vector DB using Sentence Transformations**

In [9]:
  # construct vector store and customize storage context
storage_context = StorageContext.from_defaults(
    vector_store=PineconeVectorStore(pc.Index("nagp",host="https://nagp-qtm90cl.svc.aped-4627-b74a.pinecone.io"))
)
index = VectorStoreIndex(filtered_nodes, storage_context=storage_context,embed_model=embed_model)



Upserted vectors:   0%|          | 0/30 [00:00<?, ?it/s]

**Grabbing Top 2 Nodes based on Similarity Search**

In [10]:
  from llama_index.core.retrievers import VectorIndexRetriever
  # Grab 5 search results
  start_time = time.time()
  retriever = VectorIndexRetriever(index=index, similarity_top_k=2)

  # # Query vector DB
  answer = retriever.retrieve('Who is Annadata?')
  time_elapsed = time.time() - start_time
  print("total time taken for response in seconds ",time_elapsed)
  # # Inspect results
  print([i.get_content() for i in answer])

total time taken for response in seconds  1.6001760959625244
['through avoidance of leakages prevalent earlier. The savings\nhave helped in providing more funds for ‘Garib Kalyan’.\n\n12.     PM-SVANidhi has provided credit assistance to 78 lakh\nstreet vendors. From that total, 2.3 lakh have received credit for\nthe third time.\n\n13.     PM-JANMAN Yojana reaches out to the particularly\nvulnerable tribal groups, who have remained outside the realm\nof development so far. PM-Vishwakarma Yojana provides end-to-\nend support to artisans and craftspeople engaged in 18 trades.\nThe schemes for empowerment of Divyangs and Transgender\npersons reflect firm resolve of our Government to leave no one\nbehind.\n\nWelfare of Annadata\n\n14.     Farmers are our ‘Annadata’. Every year, under PM-KISAN\nSAMMAN Yojana, direct financial assistance is provided to\n11.8 crore farmers, including marginal and small farmers. Crop\ninsurance is given to 4 crore farmers under PM Fasal Bima\nYojana. These, be

**Passing Retrieved Nodes along with query to LLM to generate response**

In [11]:
from llama_index.core.query_engine import RetrieverQueryEngine
# Pass in your retriever from above, which is configured to return the top 5 results
query_engine = RetrieverQueryEngine(retriever=retriever)

# Now you query:
start_time = time.time()
llm_query = query_engine.query('Who is Annadata')
time_elapsed = time.time() - start_time
print(llm_query.response)
print("total time taken for response in seconds ",time_elapsed)
print(llm_query.source_nodes)



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Farmers
total time taken for response in seconds  1.8830699920654297
[NodeWithScore(node=TextNode(id_='04e9b667-a042-474b-9710-50e23e5b2844', embedding=[-0.0226295236, 0.0298896395, -0.0382811911, 0.00816734228, 0.0156704783, -0.0112882489, 0.00276189647, 0.00998279639, 0.0202143, 0.0247909483, 0.0709287226, 0.0565804727, 0.0325542092, 0.0981184244, 0.0102986498, -0.0189178586, 0.00546765327, 0.00958351698, -0.0701224282, 0.0307237059, 0.013664186, 0.0030381137, -0.0121530341, 0.0473320261, -0.0205011051, -0.00165152922, 0.0342921354, -0.00322336, -0.0408226214, -0.0656796619, 0.0411885977, -0.0363950804, 0.018351946, -0.0539356209, 2.68899566e-06, -0.0106131779, -0.0104250489, 0.0203507654, -0.0759052, 0.044929307, -0.071690686, -0.0592858456, -0.00244879327, 0.0211737789, 0.00295838527, 0.0177803691, 0.0299890321, 0.0497133583, 0.00662437081, 0.0515493304, 0.0138070425, -0.0416335, -0.00723510841, 0.0293884724, -0.0446869023, 0.03219207, 0.0117866425, -0.0474332534, 0.0170690827, 0.0

**Saving Chat Context with 1000 Token Limit**

In [13]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1000)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=("You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."))


In [14]:

response = chat_engine.chat("Who is Annadata?")
print(response)

<|assistant|>
Annadata is a term used in the Indian context to refer to farmers. It is a term coined by the Indian government to describe the agricultural sector in India, which includes farmers, agricultural workers, and agricultural producers. Annadata is a crucial component of the Indian economy, and the government's efforts to support and empower this sector have been a key focus of its policies and programs.


In [15]:
response = chat_engine.chat("What are the schemes available for them?")
print(response)

<|assistant|>
Yes, Annadata is a crucial component of the Indian economy, and the government's policies and programs have been designed to support and empower this sector. Here are some of the schemes available for Annadata:

1. Pradhan Mantri Fasal Bima Yojana (PMFBY): This scheme provides insurance coverage to farmers against crop losses due to natural calamities such as floods, droughts, and cyclones.

2. Pradhan Mantri Krishi Sinchai Yojana (PMKSY): This scheme provides financial assistance to farmers for various agricultural activities such as soil health management, crop insurance, and irrigation.

3. National Agricultural Insurance Scheme (NAIS): This scheme provides insurance coverage to farmers against crop losses due to pests, diseases, and other adverse weather conditions.

4. National Agricultural Market Scheme (NAMS): This scheme aims to promote the development of agricultural markets and facilitate the sale and purchase of agricultural products.

5. National Agricultural 

In [16]:
response = chat_engine.chat("What does they do for our country?")
print(response)

<|assistant|>
Yes, Annadata plays a crucial role in the development of the Indian economy. Here are some ways in which they contribute to the country:

1. Agriculture: Annadata plays a significant role in the agricultural sector of the country. They provide financial assistance to farmers for various agricultural activities such as soil health management, crop insurance, and irrigation. This helps in improving the productivity of agricultural land and increasing the income of farmers.

2. Livestock: Annadata also plays a significant role in the livestock sector of the country. They provide financial assistance to farmers for the purchase of livestock, which helps in increasing the productivity of livestock farming.

3. Fisheries: Annadata plays a crucial role in the fisheries sector of the country. They provide financial assistance to fishermen for the purchase of fishing gear and equipment, which helps in increasing the productivity of fishing.

4. Forestry: Annadata plays a significa

In [17]:
response = chat_engine.chat("How government is supporting for their welfare?")
print(response)

<|assistant|>
The Indian government is taking several measures to support the welfare of the poor and marginalized sections of society. Here are some of the ways in which they are doing so:

1. Direct Benefit Transfer (DBT): The government has implemented Direct Benefit Transfer (DBT) to provide financial assistance to the poor and marginalized sections of society. DBT involves transferring subsidies directly to the beneficiaries' bank accounts, which helps in reducing the burden of cash transfers and improving the efficiency of the system.

2. Pradhan Mantri Jan Dhan Yojana (PMJDY): The PMJDY is a financial inclusion program launched by the government to provide bank accounts to the poor and marginalized sections of society. The program aims to provide financial services to the unbanked and underbanked population, which helps in reducing poverty and improving the quality of life of the beneficiaries.

3. National Rural Livelihood Mission (NRLM): The NRLM is a government-sponsored prog

In [18]:
response = chat_engine.chat("Which scheme is helping them more for growth of country's economy?")
print(response)

<|assistant|>
Both the Annadata and the government-sponsored schemes are crucial for the growth of the Indian economy. Here are some of the key schemes that are helping the country's economy:

1. Annadata: Annadata plays a significant role in the agricultural sector of the country. The government's Annadata scheme provides financial assistance to farmers for various agricultural activities such as soil health management, crop insurance, and irrigation. This helps in improving the productivity of agricultural land and increasing the income of farmers.

2. Pradhan Mantri Fasal Bima Yojana (PMFBY): The PMFBY is a government-sponsored scheme aimed at providing insurance coverage to farmers against crop losses due to natural calamities. The scheme provides financial assistance to farmers in the event of crop losses, which helps in reducing the financial burden on farmers and improving their livelihoods.

3. National Rural Livelihood Mission (NRLM): The NRLM is a government-sponsored program

In [19]:
response = chat_engine.chat("How we define them?")
print(response)


<|assistant|>
The Skill India Mission is a government-sponsored initiative aimed at increasing the skill-level of the Indian workforce. The mission aims to provide vocational and technical education, skill development programs, and job opportunities to the youth in the country. Here are some of the key objectives of the Skill India Mission:

1. Increase the skill-level of the Indian workforce: The mission aims to increase the skill-level of the Indian workforce by providing vocational and technical education, skill development programs, and job opportunities.

2. Promote entrepreneurship: The mission aims to promote entrepreneurship by providing financial assistance, mentorship, and support to startups and small businesses.

3. Increase employment opportunities: The mission aims to increase employment opportunities by providing job opportunities in various sectors, including manufacturing, agriculture, and service sectors.

4. Promote skill transfer: The mission aims to promote skill t

In [20]:
response = chat_engine.chat("What does The Skill India Mission do?")
print(response)

<|assistant|>
In the context of the Skill India Mission, the term "skill" refers to the ability to perform a specific task or function. The Skill India Mission aims to develop a skilled workforce by providing vocational and technical education, skill development programs, and job opportunities. The term "skill" can be defined in various ways, depending on the context. Here are some common definitions:

1. Skill development: The Skill India Mission aims to develop a skilled workforce by providing vocational and technical education, skill development programs, and job opportunities. This definition includes the development of specific skills, such as computer literacy, English proficiency, or technical skills.

2. Skill transfer: The Skill India Mission aims to promote skill transfer by providing training and certification programs to skilled workers. This definition includes the transfer of skills from one person to another, such as a skilled worker transferring their skills to a new jo

**EVALUATING PERFORMANCE OF MODEL**

**1. Response Evaluation**

In [28]:
from llama_index.core.evaluation import FaithfulnessEvaluator


# define evaluator
evaluator = FaithfulnessEvaluator(llm=llm)


# query index
query_engine = index.as_query_engine()
response = query_engine.query(
    "What is the estimate of the fiscal deficit?"
)
print(response)
eval_result = evaluator.evaluate_response(response=response)
print("Is the response matching the sources?")
print(str(eval_result.passing))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


5.8% of GDP, improving on the Budget Estimate, notwithstanding moderation in the nominal growth estimates.
Is the response matching the sources?
True


**2. Retriever Evaluation**

In [30]:
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.core.evaluation import generate_question_context_pairs

qa_dataset = generate_question_context_pairs(
    filtered_nodes, llm=llm, num_questions_per_chunk=2
)
retriever_evaluator = RetrieverEvaluator.from_metric_names(["mrr", "hit_rate"], retriever=retriever)
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)



  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 47%|████▋     | 14/30 [01:44<01:24,  5.31s/it]/usr/local/lib/python3.10/dist-packages/llama_index/core/llama_dataset/legacy/embedding.py:99: UserWarning: Fewer questions generated (1) than requested (2).
  warnings.warn(
100%|██████████| 30/30 [03:45<00:00,  7.52s/it]


In [31]:
print(eval_results)

[RetrievalEvalResult(query="Question 1: What is the government's plan to increase the minimum wage for workers in the country?", expected_ids=['61269ca0-b271-4c27-a05e-4ff8a70d1649'], expected_texts=None, retrieved_ids=['2e73cc5f-c948-4bbf-8c41-03036304d56f', 'dd516494-41fa-44e8-90a4-142f3e7d6de7'], retrieved_texts=['Part B\n\nHon’ble Speaker Sir,\n\nDirect taxes\n\n87.     Over the last ten years, the direct tax collections have\nmore than trebled and the return filers swelled to 2.4 times.\nI would like to assure the taxpayers that their contributions have\nbeen used wisely for the development of the country and\nwelfare of its people. I appreciate the tax payers for their\nsupport.\n\n88.     The Government has reduced and rationalized tax rates.\nUnder the new tax scheme, there is now no tax liability for tax\npayers with income up to   7 lakh, up from   2.2 lakh in the\nfinancial year 2013-14. The threshold for presumptive taxation\nfor retail businesses was increased from   2 cro